# Testing Playground for r5py

In [1]:
import r5py

## Sample Datasets Provided by r5py themselves

### Helsinki

In [2]:
import r5py.sampledata.helsinki
import geopandas
import shapely

In [3]:
population_grid = geopandas.read_file(r5py.sampledata.helsinki.population_grid)

railway_station = geopandas.GeoDataFrame(
    {
        "id": ["railway_station"],
        "geometry": [shapely.Point(24.94152, 60.17066)]
    },
    crs="EPSG:4326",
)

In [4]:
overview_map = population_grid.explore("population", cmap="Reds")
overview_map = railway_station.explore(m=overview_map, marker_type="marker")
overview_map

In [5]:
transport_network = r5py.TransportNetwork(
    r5py.sampledata.helsinki.osm_pbf,
    [r5py.sampledata.helsinki.gtfs],
)

In [6]:
import datetime

origins = population_grid.copy()
origins.geometry = origins.geometry.centroid

destinations = railway_station.copy()

travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    departure=datetime.datetime(2022, 2, 22, 8, 30),
    transport_modes=[
        r5py.TransportMode.TRANSIT,
        r5py.TransportMode.WALK,
    ],
)

/tmp/ipykernel_18470/3435707801.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  origins.geometry = origins.geometry.centroid


In [7]:
travel_times = travel_time_matrix_computer.compute_travel_times()
travel_times.head()

,from_id,to_id,travel_time
0,0,railway_station,16
1,1,railway_station,18
2,2,railway_station,20
3,3,railway_station,22
4,4,railway_station,18


In [8]:
travel_times = population_grid.merge(travel_times, left_on="id", right_on="from_id")
travel_times.head()

,id,population,geometry,from_id,to_id,travel_time
0,0,389,"POLYGON ((24.90545 60.16086, 24.90545 60.16311...",0,railway_station,16
1,1,296,"POLYGON ((24.90546 60.15862, 24.90545 60.16086...",1,railway_station,18
2,2,636,"POLYGON ((24.90547 60.15638, 24.90546 60.15862...",2,railway_station,20
3,3,1476,"POLYGON ((24.90547 60.15413, 24.90547 60.15638...",3,railway_station,22
4,4,23,"POLYGON ((24.90994 60.16535, 24.90994 60.16760...",4,railway_station,18


In [9]:
travel_times.explore("travel_time", cmap="Greens")